In [7]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import ast
from scipy.stats import ttest_ind
import seaborn as sns

In [9]:
parquet_file = 'C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/LG/데이터/postech_company_project.parquet'
# Parquet 파일 열기
table = pq.read_table(parquet_file)

# Table에서 데이터를 DataFrame으로 변환
df = table.to_pandas()
df.head()

,YEAR_YYYY,MONTH_YYYYMM,WEEK_YYYYWWW,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,PARTS_NO1,...,KEY_PARTS_REMARK3,KEY_PARTS_REMARK4,KEY_PARTS_REMARK5,PREV_KEY_PARTS_REMARK1,PREV_KEY_PARTS_REMARK2,PREV_KEY_PARTS_REMARK3,PREV_KEY_PARTS_REMARK4,PREV_KEY_PARTS_REMARK5,MULTICLAIM,PREV_RECEIPT_SYMPTOM
0,2022,202201,2022W05,47.0,LGEAI,Non Captive,HE,LTV,OLED65C7P,LGE_PARTS_993,...,Power,None,None,None,None,None,None,None,2,Power
1,2022,202201,2022W03,10.0,LGEAI,Non Captive,HE,LTV,65UN6955ZUF,LGE_PARTS_4492,...,None,None,None,Main,None,None,None,None,2,Audio
2,2022,202201,2022W05,30.0,LGEAI,Non Captive,HE,LTV,50UN7300AUD,LGE_PARTS_15608,...,None,None,None,Main,None,None,None,None,2,Power
3,2022,202201,2022W02,39.0,LGEAI,Non Captive,HE,LTV,70UP7070PUE,LGE_PARTS_9199,...,None,None,None,Power,Main,None,None,None,2,Power
4,2022,202201,2022W03,5.0,LGEAI,Non Captive,HE,LTV,OLED77CXAUA,LGE_PARTS_15413,...,None,None,None,None,None,None,None,None,2,Power


In [ ]:
print(df.shape)

(13881284, 71)


In [ ]:
df.columns

Index(['YEAR_YYYY', 'MONTH_YYYYMM', 'WEEK_YYYYWWW', 'RECLAIM_PERIOD',
       'SUBSIDIARY_NAME', 'CAPTIVE_CHANNEL_FLAG', 'COMPANY_CODE',
       'DIVISION_NAME', 'MODEL_CODE', 'PARTS_NO1', 'PARTS_NO2', 'PARTS_NO3',
       'PARTS_NO4', 'PARTS_NO5', 'PREV_PARTS_NO1', 'PREV_PARTS_NO2',
       'PREV_PARTS_NO3', 'PREV_PARTS_NO4', 'PREV_PARTS_NO5', 'PARTS_DESC1',
       'PARTS_DESC2', 'PARTS_DESC3', 'PARTS_DESC4', 'PARTS_DESC5',
       'PREV_PARTS_DESC1', 'PREV_PARTS_DESC2', 'PREV_PARTS_DESC3',
       'PREV_PARTS_DESC4', 'PREV_PARTS_DESC5', 'RECEIPT_SYMPTOM',
       'RECEIPT_NUMBER', 'PREV_RECEIPT_NUMBER', 'REPAIR_START_DATE',
       'REPAIR_END_DATE', 'CENTER_CODE', 'PREV_CENTER_CODE', 'PRODUCT_NAME',
       'PRODUCT_NAME_DETAIL', 'RECLAIM', 'WARRANTY_TYPE', 'PREV_WARRANTY_TYPE',
       'SVC_TYPE', 'PREV_SVC_TYPE', 'ENGINEER_CODE', 'PREV_ENGINEER_CODE',
       'ENGINEER_SYMPTOM', 'ENGINEER_REPAIR', 'PREV_ENGINEER_SYMPTOM',
       'PREV_ENGINEER_REPAIR', 'KEY_PARTS1', 'KEY_PARTS2', 'KEY_PARTS3

# ENGINEER _REPAIR 분석

### 상위 10개만 남기자

In [15]:
df[['ENGINEER_REPAIR']].nunique()

ENGINEER_REPAIR    176
dtype: int64

In [11]:
df[['ENGINEER_REPAIR']].value_counts()

ENGINEER_REPAIR                       
Installation of new product               4006602
Demo given                                2412943
Parts replaced                            1569328
Internal adjustment                        843721
Wet/Preventive Maintenance                 799063
                                           ...   
LED PKG Exchange_Pattern Repair                 1
BIOS Setup changed                              1
Re Gas Charge                                   1
Modification requested by manufacturer          1
Clean set externally                            1
Name: count, Length: 176, dtype: int64

In [22]:
# 'ENGINEER_REPAIR' 컬럼 값의 빈도를 조사한다
repair_counts = df['ENGINEER_REPAIR'].value_counts()

# 상위 100개 값을 추려내기
top_100_repairs = repair_counts.head(10).index

# 'ENGINEER_REPAIR' 컬럼 값이 상위 100개에 해당하는 관측치만 추출한다
filtered_df = df[df['ENGINEER_REPAIR'].isin(top_100_repairs)]

# 결과를 출력한다
print(filtered_df.shape)
filtered_df

(11700700, 71)


,YEAR_YYYY,MONTH_YYYYMM,WEEK_YYYYWWW,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,PARTS_NO1,...,KEY_PARTS_REMARK3,KEY_PARTS_REMARK4,KEY_PARTS_REMARK5,PREV_KEY_PARTS_REMARK1,PREV_KEY_PARTS_REMARK2,PREV_KEY_PARTS_REMARK3,PREV_KEY_PARTS_REMARK4,PREV_KEY_PARTS_REMARK5,MULTICLAIM,PREV_RECEIPT_SYMPTOM
223,2022,202201,2022W04,NaN,LGEAI,Non Captive,HE,LTV,43UN6950ZUA,LGE_PARTS_31555,...,None,None,None,None,None,None,None,None,1,None
243,2022,202201,2022W04,NaN,LGEAI,Non Captive,HE,LTV,55UP8000PUA,LGE_PARTS_6255,...,None,None,None,None,None,None,None,None,1,None
261,2022,202201,2022W01,NaN,LGEAI,Non Captive,HE,LTV,70UN7070PUA,LGE_PARTS_21895,...,None,None,None,None,None,None,None,None,1,None
395,2022,202201,2022W02,NaN,LGEAI,Non Captive,HE,LTV,50UN6950ZUF,LGE_PARTS_8111,...,None,None,None,None,None,None,None,None,1,None
401,2022,202201,2022W01,NaN,LGEAI,Non Captive,HE,LTV,55UM6910PUC,LGE_PARTS_28203,...,None,None,None,None,None,None,None,None,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14196328,2023,202306,2023W25,NaN,LGEIL,Non Captive,HA,RAC DIVISION,RS-Q19KWZE,None,...,None,None,None,None,None,None,None,None,1,None
14196329,2023,202306,2023W23,NaN,LGEIL,Non Captive,HA,RAC DIVISION,RS-Q19PNZE,None,...,None,None,None,None,None,None,None,None,1,None
14196330,2023,202306,2023W23,NaN,LGEIL,Captive,HA,RAC DIVISION,RS-Q19QNZE,None,...,None,None,None,None,None,None,None,None,1,None
14196331,2023,202306,2023W23,NaN,LGEIL,Non Captive,HA,RAC DIVISION,RS-Q19QNZE,None,...,None,None,None,None,None,None,None,None,1,None


# 부품 상위 추출

In [36]:
# 부품을 교체하지 않은 데이터 삭제
df1 = df.dropna(subset=['PARTS_NO1'])

print(df.shape)
print(df1.shape)

(13881284, 71)
(5057164, 71)


In [30]:
parts_df = df1[['PARTS_NO1','PARTS_NO2','PARTS_NO3','PARTS_NO4','PARTS_NO5']]
parts_df

,PARTS_NO1,PARTS_NO2,PARTS_NO3,PARTS_NO4,PARTS_NO5
0,LGE_PARTS_993,LGE_PARTS_18111,LGE_PARTS_20442,None,None
1,LGE_PARTS_4492,None,None,None,None
2,LGE_PARTS_15608,None,None,None,None
3,LGE_PARTS_9199,None,None,None,None
4,LGE_PARTS_15413,None,None,None,None
...,...,...,...,...,...
14196312,LGE_PARTS_32802,None,None,None,None
14196315,LGE_PARTS_27082,None,None,None,None
14196316,LGE_PARTS_27082,None,None,None,None
14196317,LGE_PARTS_28382,None,None,None,None


In [32]:
# PARTS_NO# 별 관측치 비율
(parts_df.isnull().mean() * 100).round(2)

PARTS_NO1     0.00
PARTS_NO2    77.59
PARTS_NO3    91.62
PARTS_NO4    96.03
PARTS_NO5    98.07
dtype: float64

In [40]:
# PARTS_NO# 별 관측치 비율
(df1[['PARTS_NO1']].isnull().mean() * 100).round(2)

PARTS_NO1    0.0
dtype: float64

In [33]:
parts_df[['PARTS_NO1']].value_counts()

PARTS_NO1      
LGE_PARTS_27082    1472962
LGE_PARTS_13404     322128
LGE_PARTS_33913     119270
LGE_PARTS_5313       96835
LGE_PARTS_25288      86088
                    ...   
LGE_PARTS_13861          1
LGE_PARTS_3442           1
LGE_PARTS_1684           1
LGE_PARTS_13863          1
LGE_PARTS_34485          1
Name: count, Length: 27960, dtype: int64

In [50]:
# 'PARTS_NO1' 컬럼 값의 빈도를 조사한다
parts_counts = df1['PARTS_NO1'].value_counts()

# 상위 100개 값을 추려내기
top_100_parts = parts_counts.head(100).index

# 'PARTS_NO1' 컬럼 값이 상위 100개에 해당하는 관측치만 추출한다
filtered_df = df1[df1['PARTS_NO1'].isin(top_100_parts)]

# 결과를 출력한다
print(filtered_df.shape)
filtered_df

(3432863, 71)


,YEAR_YYYY,MONTH_YYYYMM,WEEK_YYYYWWW,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,PARTS_NO1,...,KEY_PARTS_REMARK3,KEY_PARTS_REMARK4,KEY_PARTS_REMARK5,PREV_KEY_PARTS_REMARK1,PREV_KEY_PARTS_REMARK2,PREV_KEY_PARTS_REMARK3,PREV_KEY_PARTS_REMARK4,PREV_KEY_PARTS_REMARK5,MULTICLAIM,PREV_RECEIPT_SYMPTOM
48,2022,202201,2022W04,NaN,LGEAI,Non Captive,HE,LTV,43UN6955ZUF,LGE_PARTS_14916,...,None,None,None,None,None,None,None,None,1,None
208,2022,202201,2022W01,26.0,LGEAI,Non Captive,HE,LTV,OLED65CXAUA,LGE_PARTS_26909,...,None,Main,None,Module,None,None,None,None,2,Video
209,2022,202201,2022W02,-10.0,LGEAI,Non Captive,HE,LTV,OLED65CXPUA,LGE_PARTS_26909,...,None,None,None,Module,None,None,None,None,2,Video
460,2022,202201,2022W04,NaN,LGEAI,Non Captive,HE,LTV,OLED55CXPUA,LGE_PARTS_26909,...,None,None,None,None,None,None,None,None,1,None
652,2022,202201,2022W02,NaN,LGEAI,Non Captive,HE,LTV,OLED65A1PUA,LGE_PARTS_26909,...,None,None,None,None,None,None,None,None,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14196298,2023,202306,2023W23,NaN,LGEIL,Captive,HE,LTV,32LQ570BPSA,LGE_PARTS_27082,...,None,None,None,None,None,None,None,None,1,None
14196315,2023,202306,2023W23,NaN,LGEIL,Non Captive,BS,Commercial TV,32LU640H,LGE_PARTS_27082,...,None,None,None,None,None,None,None,None,1,None
14196316,2023,202306,2023W24,NaN,LGEIL,Captive,BS,Signage,43UH5J-H,LGE_PARTS_27082,...,None,None,None,None,None,None,None,None,1,None
14196317,2023,202306,2023W23,NaN,LGEIL,Captive,HA,HEALTHCARE,WW140NPR,LGE_PARTS_28382,...,None,None,None,None,None,None,None,None,1,None


In [51]:
filtered_df.to_csv('C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/filtered_df.csv')